For røykerandelen av befolkningen
https://www.ssb.no/helse/helseforhold-og-levevaner/statistikk/royk-alkohol-og-andre-rusmidler

For luftkvalitet
https://luftkvalitet.nilu.no/historikk



Potensielt 
https://norgeshelsa.no/norgeshelsa/

## Datakilde
- statbank
- asthma og status asthmaticus (astma og akutt astma)
- per nå for alle år, alle kjønn, alle aldere (gruppevis) for både liggedager, dagbehandlinger og utksrivninger.

https://www.ssb.no/en/statbank/table/03672/tableViewLayout1/

In [3]:
import pandas as pd
import numpy as np
df = pd.read_excel("processed_data.xlsx")
df.head(15)

,Age group,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008
0,"Years, total",7.0,6.4,6.6,4.9,4.2,3.8,3.5,4.9,4.9,5.2
1,0-9 years,15.8,11.6,12.5,8.5,7.7,7.2,6.9,9.6,8.4,7.8
2,10-19 years,10.5,11.4,12.1,2.0,1.6,1.5,1.3,6.6,7.1,8.9
3,20-39 years,3.0,2.5,2.4,2.5,2.0,2.0,1.7,1.7,1.7,1.8
4,40-59 years,4.6,4.8,5.9,5.3,4.6,4.8,4.2,4.3,4.5,5.1
5,60-69 years,6.5,6.1,7.1,7.5,5.9,4.2,4.6,5.9,6.4,6.1
6,70-79 years,8.9,8.4,6.5,8.3,7.0,5.0,4.6,6.7,6.2,6.8
7,80 years or older,6.6,7.4,4.7,6.1,5.3,3.6,3.8,4.7,4.6,4.0
8,Unknown age,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,"Years, total",6.7,5.5,5.3,5.3,4.4,4.2,3.5,4.5,6.6,5.5


In [4]:

all_rows = ["Total", "0-9 years", "10-19 years", "20-39 years",
            "40-59 years", "60-69 years", "70-79 years",
            "80 years or older", "Unknown age"]


In [5]:
age_col = None
if isinstance(df.columns, pd.MultiIndex):
    for col in df.columns:
        # try to detect a column whose values match your age labels
        if df[col].astype(str).str.strip().isin(all_rows).any():
            age_col = col
            break
else:
    # common single-level names
    for cand in ("Age group", "age group", "age", "group"):
        if cand in df.columns:
            age_col = cand
            break
    # fallback: any string column that overlaps labels
    if age_col is None:
        for c in df.columns:
            if df[c].dtype == object and df[c].astype(str).str.strip().isin(all_rows).any():
                age_col = c
                break

In [6]:
if age_col is None:
    raise KeyError("Could not find a column containing the age-group labels")

# normalized label column used for grouping
df["_age_label"] = df[age_col].astype(str).str.strip()

# sum numeric columns per age label
numeric_cols = df.select_dtypes(include=[np.number]).columns
grouped = df.groupby("_age_label")[numeric_cols].sum()

# ensure the desired order and fill missing with 0
grouped = grouped.reindex(all_rows[1:], fill_value=0)

# create Total row (sum of all groups) and prepend
total = grouped.sum(axis=0).to_frame().T
total.index = ["Total"]
result = pd.concat([total, grouped])

In [7]:
display(result)

,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008
Total,1073.7,1006.9,987.3,835.8,673.8,572.3,548.3,745.8,746.5,747.6
0-9 years,290.9,212.0,223.1,158.2,143.5,143.2,129.9,179.1,160.7,142.6
10-19 years,176.9,188.6,198.2,38.5,31.4,29.4,28.5,101.3,109.4,136.0
20-39 years,66.1,51.4,48.7,52.9,46.3,41.2,36.1,37.2,37.4,42.2
40-59 years,91.0,104.4,125.4,116.3,100.6,98.6,87.5,85.4,92.0,103.1
60-69 years,134.6,131.8,153.3,166.2,119.9,88.5,92.0,124.6,131.4,127.3
70-79 years,181.2,171.5,150.4,158.1,132.2,98.4,102.3,127.7,125.5,124.6
80 years or older,133.0,147.2,88.2,145.6,99.9,73.0,72.0,90.5,90.1,71.8
Unknown age,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
# Denmark data
df_den = pd.read_excel('denmark1991_2005.xlsx')
df_den.head(15)

,"Admissions by diagnosis, age and time",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,Units: Number,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,1991.0,1992.0,1993.0,1994.0,1995.0,1996.0,1997.0,1998.0,1999.0,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0
2,"Bronchitis, emphysema and asthma","Age, total",18631.0,19657.0,20642.0,26881.0,29249.0,30867.0,33001.0,34603.0,35240.0,34604.0,35823.0,32862.0,31196.0,32146.0,31995.0
3,NaN,0 years,0.0,0.0,0.0,0.0,667.0,658.0,1084.0,931.0,883.0,713.0,869.0,842.0,568.0,1048.0,831.0
4,NaN,1-4 years,901.0,1059.0,1099.0,4715.0,4855.0,5853.0,5143.0,5430.0,5348.0,5618.0,6230.0,5777.0,5788.0,6934.0,6868.0
5,NaN,5-9 years,790.0,732.0,731.0,648.0,765.0,714.0,831.0,777.0,887.0,928.0,1164.0,1044.0,907.0,1157.0,1085.0
6,NaN,10-14 years,469.0,464.0,397.0,379.0,370.0,274.0,361.0,330.0,380.0,362.0,431.0,429.0,438.0,508.0,495.0
7,NaN,15-19 years,373.0,319.0,261.0,217.0,232.0,223.0,237.0,181.0,212.0,201.0,205.0,178.0,214.0,206.0,231.0
8,NaN,20-24 years,321.0,282.0,257.0,257.0,302.0,309.0,259.0,277.0,227.0,187.0,245.0,216.0,194.0,187.0,212.0
9,NaN,25-29 years,332.0,273.0,260.0,284.0,280.0,304.0,255.0,249.0,253.0,339.0,294.0,283.0,278.0,286.0,207.0


In [ ]:
# Denmark asthma data 1991-2005
df_den = pd.read_excel('denmark1991_2005.xlsx')
df_den.head(15)
df_den.columns = df_den.iloc[1]
df_den = df_den.iloc[2:].reset_index(drop=True)
df_den = df_den.rename(columns={df_den.columns[1]: 'age_group'})
df_den.set_index('age_group')


df_den.columns = df_den.columns.astype(str)

df_den.head(15)

1,age_group,age_group,1991.0,1992.0,1993.0,1994.0,1995.0,1996.0,1997.0,1998.0,1999.0,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0
0,"Bronchitis, emphysema and asthma","Age, total",18631.0,19657.0,20642.0,26881.0,29249.0,30867.0,33001.0,34603.0,35240.0,34604.0,35823.0,32862.0,31196.0,32146.0,31995.0
1,NaN,0 years,0.0,0.0,0.0,0.0,667.0,658.0,1084.0,931.0,883.0,713.0,869.0,842.0,568.0,1048.0,831.0
2,NaN,1-4 years,901.0,1059.0,1099.0,4715.0,4855.0,5853.0,5143.0,5430.0,5348.0,5618.0,6230.0,5777.0,5788.0,6934.0,6868.0
3,NaN,5-9 years,790.0,732.0,731.0,648.0,765.0,714.0,831.0,777.0,887.0,928.0,1164.0,1044.0,907.0,1157.0,1085.0
4,NaN,10-14 years,469.0,464.0,397.0,379.0,370.0,274.0,361.0,330.0,380.0,362.0,431.0,429.0,438.0,508.0,495.0
5,NaN,15-19 years,373.0,319.0,261.0,217.0,232.0,223.0,237.0,181.0,212.0,201.0,205.0,178.0,214.0,206.0,231.0
6,NaN,20-24 years,321.0,282.0,257.0,257.0,302.0,309.0,259.0,277.0,227.0,187.0,245.0,216.0,194.0,187.0,212.0
7,NaN,25-29 years,332.0,273.0,260.0,284.0,280.0,304.0,255.0,249.0,253.0,339.0,294.0,283.0,278.0,286.0,207.0
8,NaN,30-34 years,257.0,244.0,225.0,264.0,241.0,275.0,358.0,318.0,274.0,323.0,316.0,310.0,327.0,364.0,388.0
9,NaN,35-39 years,322.0,294.0,284.0,327.0,322.0,280.0,332.0,379.0,400.0,373.0,417.0,381.0,385.0,435.0,457.0
